In [2]:
from bulk_simulation_helper import *

In [ ]:
out_fn = make_file_name( 'full_run_3' )

In [14]:
strats = [
    sampling.friendHighDegClose,
    sampling.friendClose,
]

In [17]:
VACC_Ps = [0.05, 0.10, 0.20]

In [17]:
argsx = cp(
    [0.05, 0.1, 0.2], # S2E's
    [0,1] # tneti's
)

In [18]:
threads = []
for S2E, tneti in argsx:
    t = Thread(target=start_worker, args=(S2E, tneti, VACC_Ps, strats))
    t.start()
    threads.append(t)
        
for t in threads:
    t.join()

loading network...
loading network...
loading network...
loading network...
loading network...
loading network...
Starting on  (0, 0.1, 0.05, 'friendHighDegClose')
Starting on  (0, 0.2, 0.05, 'friendHighDegClose')
Starting on  (0, 0.05, 0.05, 'friendHighDegClose')
simulation 20
simulation 20
simulation 20
simulation 40
simulation 40
simulation 40
Starting on  (1, 0.1, 0.05, 'friendHighDegClose')
Starting on  (1, 0.2, 0.05, 'friendHighDegClose')
simulation 60
simulation 60
simulation 60
simulation 80
Starting on  (1, 0.05, 0.05, 'friendHighDegClose')
simulation 100
simulation 80
simulation 80
simulation 120
simulation 100
simulation 100
simulation 140
simulation 160
simulation 120
simulation 120
simulation 180
simulation 200
Starting on  (0, 0.2, 0.05, 'friendClose')
simulation 140
simulation 140
simulation 20
simulation 160
simulation 160
simulation 40
simulation 60
simulation 180simulation 180

simulation 80
simulation 100
simulation 200
Starting on simulation 200 
(0, 0.05, 0.05, 'fr

In [30]:
len(ms)

36

In [20]:
len(ms[list(ms.keys())[0]])

200

In [22]:
if True:
    import pickle
    with open(outfn, 'wb') as outf:
        pickle.dump( ms, outf )